In [1]:
from willitfit.scrapers.IKEA import product_info_and_update_csv_database

article_code = ['s39128564','90499066','s29178074','40258154','30269129']






product_info_and_update_csv_database(article_code)







[['s39128564', 1, [0, 49.0, 6.0, 4.0, 0.28]],
 ['s39128564', 1, [0, 49.0, 6.0, 4.0, 0.28]],
 ['s39128564', 1, [0, 74.0, 6.0, 4.0, 0.43]],
 ['s39128564', 1, [0, 74.0, 6.0, 4.0, 0.43]],
 ['s39128564', 1, [0, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [1, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [2, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [3, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [4, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [5, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [0, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [1, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [2, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [3, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [4, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [5, 59.0, 48.0, 2.0, 3.35]],
 ['s39128564', 1, [0, 245.0, 61.0, 7.0, 44.16]],
 ['s39128564', 1, [1, 245.0, 61.0, 7.0, 44.16]],
 ['90499066', 1, [0, 96.0, 78.0, 65.0, 41.2]],
 ['s29178074', 1, [0, 59.0, 58.0, 55.0, 6.7]],
 ['s29178074', 1, [0, 59.0, 58.0, 55.0, 6.7]],
 ['s29178074'